In [1]:
import pandas as pd
from pymongo import MongoClient
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
import warnings; warnings.simplefilter('ignore')
import pickle

def flatten (lst):
    return [el for sub in lst for el in sub]

In [2]:
conn = MongoClient('localhost', 27017)
db = conn.forca_ia
collection = db.verbetes

In [3]:
df = pd.DataFrame(list(collection.find()))
#df =  df[:1000]

lis = [el for el in flatten(df['fill_word']) if el[-1] not in el[1] if "_" in el[1]]

In [4]:
df2 = pd.DataFrame(lis, columns = ["isTrue", "bef_pattern", "aft_pattern", "letter_choice"])
#df2 = df2[:150000]

y = df2['isTrue']
x = [df2['bef_pattern'].iloc[lin]+", "+df2['letter_choice'].iloc[lin] for lin in range(len(y))]

In [5]:
with open("x_list2", "wb") as file:
    pickle.dump(x, file)
    file.close()

In [6]:
tokenizer = Tokenizer(lower = False, filters=' ', char_level=False, split=',')
tokenizer.fit_on_texts(x)
tok1 = tokenizer.texts_to_sequences(x)
tok_df = pd.DataFrame(data = tok1).fillna(0.0)

y.to_pickle('Y2.pkl')
tok_df.to_pickle('X2.pkl')

In [8]:
y = pd.read_pickle('Y2.pkl')
x = pd.read_pickle('X2.pkl')

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.3)

In [10]:
dtree = DecisionTreeClassifier()
dtree.fit(X_train, Y_train)
dtree_pred = dtree.predict(X_test)

In [11]:
print(classification_report(Y_test, dtree_pred))
print('\n')
print(confusion_matrix(Y_test, dtree_pred))

              precision    recall  f1-score   support

       False       0.97      0.98      0.97   3090887
        True       0.90      0.86      0.88    733915

    accuracy                           0.95   3824802
   macro avg       0.93      0.92      0.93   3824802
weighted avg       0.95      0.95      0.95   3824802



[[3017285   73602]
 [ 101187  632728]]


In [ ]:

with open("dtree2b.pickle", "wb") as file:
    pickle.dump(dtree, file)
    file.close()
print ("Modelo salvo.")

with open('dtree2b.pickle', "rb") as file:
    dtree = pickle.load(file)
    file.close()

dtree_pred = dtree.predict(X_test)
print(classification_report(Y_test, dtree_pred))
print('\n')
print(confusion_matrix(Y_test, dtree_pred))


Modelo salvo.


In [ ]:
random.c